In [145]:
from sklearn.datasets import fetch_covtype
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (fetch_covtype())

x = pd.DataFrame(datasets["data"])
y = pd.DataFrame(datasets["target"])


In [146]:
print(x.shape,y.shape) 

print(np.unique(y)) # [0 1 2 3 4 5 6 7 8 9]
# 불균형확인
print(np.unique(y,return_counts=True)) # [0 1 2]



(581012, 54) (581012, 1)
[1 2 3 4 5 6 7]
(array([1, 2, 3, 4, 5, 6, 7]), array([211840, 283301,  35754,   2747,   9493,  17367,  20510],
      dtype=int64))


In [152]:
# print(x.info())
# print(x.describe())
# 데이터 분리
x_train, x_test, y_train,y_test = train_test_split(
                                                    x,
                                                    y,
                                                    train_size=0.8,
                                                    shuffle = True,
                                                    #  stratify 는 데이터 불균형을 해결해줌
                                                    stratify=y,
                                                    random_state=21
                                                   )
# print(x.columns)
# y 의 첫번째 칼럼을 원핫 인코딩 변환
y_train = tf.one_hot(y_train[0],depth =7)
y_test = tf.one_hot(y_test[0],depth= 7)


print(y_train[:16])
print("shape : ",x_train.shape)
print("shape : ",y_train.shape)

tf.Tensor(
[[0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]], shape=(16, 7), dtype=float32)
shape :  (464809, 54)
shape :  (464809, 7)


In [148]:
model = Sequential([
    Dense(10,activation="relu",input_shape=(54,)),
    Dense(10,activation="relu") ,
    Dense(10,activation="relu") ,
    Dense(10,activation="relu") ,
    Dense(10,activation="relu") ,
    Dense(10,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(7,activation="softmax")
])

In [149]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
patience=5, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=10, steps_per_epoch=100, 
                 validation_split=0.2,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


Epoch 1/10
100/100 [==============================] - 5s 51ms/step - loss: 35.2963 - accuracy: 0.3035 - val_loss: 2.7309 - val_accuracy: 0.4106
Epoch 2/10
100/100 [==============================] - 5s 52ms/step - loss: 1.8522 - accuracy: 0.4274 - val_loss: 1.3296 - val_accuracy: 0.4592
Epoch 3/10
100/100 [==============================] - 5s 51ms/step - loss: 1.1594 - accuracy: 0.4684 - val_loss: 1.0624 - val_accuracy: 0.4778
Epoch 4/10
100/100 [==============================] - 5s 52ms/step - loss: 1.0559 - accuracy: 0.4622 - val_loss: 1.0337 - val_accuracy: 0.4547
Epoch 5/10
100/100 [==============================] - 5s 53ms/step - loss: 1.0122 - accuracy: 0.4816 - val_loss: 0.9995 - val_accuracy: 0.4917
Epoch 6/10
100/100 [==============================] - 5s 55ms/step - loss: 0.9897 - accuracy: 0.4893 - val_loss: 0.9856 - val_accuracy: 0.4877
Epoch 7/10
100/100 [==============================] - 5s 55ms/step - loss: 0.9801 - accuracy: 0.4893 - val_loss: 0.9781 - val_accuracy: 0.488

In [150]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


3632/3632 [==============================] - 8s 2ms/step - loss: 0.9510 - accuracy: 0.4978
loss: 0.9510359168052673 
acc : 0.49780985713005066


In [ ]:
from sklearn.metrics import accuracy_score
y_predict = model.predict(x_test)

# predict 의 값은 총 3개가 나오는데 softmax 특성상 sum 의 값은 1이 됨
# print(y_predict[:5])

# argmax 는 x 를 최대값으로 만들어주는 입력을 구하는함수
y_predict = np.argmax(y_predict,axis=1)
y_test = np.argmax(y_test,axis=1)

In [160]:
# 데이터 출력

print("예측데이터 : ",y_predict[:20])
print("실제데이터 : ",y_test[:20])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print(acc)


예측데이터 :  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
실제데이터 :  [1 2 1 1 2 1 1 1 2 1 2 2 3 2 2 2 2 2 2 1]
0.49780986721513215
